<a href="https://colab.research.google.com/github/leonardo3108/IA368dd/blob/main/exercicios/Aula_5/Aula_5_Treino_Modelo_de_Linguagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Leonardo Augusto da Silva Pacheco'
print(f'Meu nome é {nome}')

Meu nome é Leonardo Augusto da Silva Pacheco


# Introdução

## Enunciado

Treinar um modelo de linguagem em dados em portugues
- Avaliar o modelo usando a perplexidade, que é simplesmente a exponencial de todas as losses do dataset de validação
- Iremos treinar o modelo para prever o próximo token dado os anteriores (também conhecido como Causal Language Modeling). Não confundir com o Masked Language Modeling (MLM), que consiste em prever tokens mascarados em uma dada sequência (ex: BERT's MLM)

Dicas:
- Usar como ponto de partida o modelo [OPT-125M](https://huggingface.co/facebook/opt-125m), que já foi treinado em 300B de tokens (maioria em Inglês)
- Usar este dataset reduzido do mc4 portugues, com ~300M de tokens: [mc4-pt-sample-1g.txt](https://unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt)


## Abordagem

Segue uma descrição dos passos utilizados e documentação dos principais conceitos:

1. Baixamos o conjunto de dados mc4-pt-sample-1g.txt e preparamos os dados para treinamento, em batchs contendo sequências de número fixo de tokens.
   - preparação do dataset: https://huggingface.co/docs/datasets/loading
   - tokens em batch: https://huggingface.co/docs/datasets/about_map_batch
2. Baixamos o modelo pré-treinado OPT-125M e efetuamos o fine-tuning usando o dataset e por meio de modelagem de dados causal.
  - treinamento: https://huggingface.co/docs/transformers/v4.27.2/en/main_classes/trainer
  - Causal language modeling with GPT: https://heartbeat.comet.ml/causal-language-modeling-with-gpt-d92c9cfe2d2a
4. Avaliamos o modelo treinado por meio da métrica perplexidade, e testamos a geração de texto (greedy decoder).
  - Perplexidade: https://huggingface.co/docs/transformers/main/en/perplexity
  - Greedy decoding: https://medium.com/geekculture/greedy-search-decoding-for-text-generation-62e6dad889b

Fontes de inspiração:
- slide da aula com o código gerado pelo ChatGPT
- notebook de exemplo da huggingface: [How to fine-tune a model on language modeling](https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb)
- notebook da [Mirelle](https://colab.research.google.com/drive/1WlSg5THKtALqYPDmqilfhjGkGHSYBJv7?usp=sharing#scrollTo=30Vu-BDcWrAa)



# Setup

## Preparação de pastas

In [2]:
!mkdir 'model_output'
!mkdir 'model_save'

## Hiperparâmetros

In [3]:
model_name = 'facebook/opt-125m'
max_seq_length=256
batch_size=16
epochs=3
model_output_dir='model_output'
model_save_dir='model_save'

## Instalação de libs

In [4]:
!pip install transformers
!pip install datasets 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.2 MB/s eta 0:00:

## Importação de libs

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
import math

## Utilização de GPU

In [6]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


In [7]:
if dev != 'cpu':
    !nvidia-smi

Wed Mar 29 01:14:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    42W / 400W |      3MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Preparação dos dados

## Obtenção

In [8]:
 !wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

--2023-03-29 01:14:38--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.68.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230909256 (1.1G) [text/plain]
Saving to: ‘sample-1gb.txt’

sample-1gb.txt      100%[===================>]   1.15G  23.0MB/s    in 52s     

2023-03-29 01:15:30 (22.8 MB/s) - ‘sample-1gb.txt’ saved [1230909256/1230909256]



### Exploração

In [9]:
!wc -l sample-1gb.txt

250000 sample-1gb.txt


In [10]:
!head -5 sample-1gb.txt

Linkbar Há alguns anos, o número de rapazes e moças que subiam ao púlpito para pregar era maior que o de hoje. Na sua simplicidade, falavam do amor de Deus, da Salvação e davam testemunho sob a unção do Espirito Santo. Hoje, parece que a figura do "preletor oficial" inibiu muitos de falarem com ousadia a Palavra de Deus. Parece que há um receio de falar diante de um público que, certamente, é mais intelectualizado que há alguns anos. Jovens pregadores ficam embaraçados e cometem certos deslizes, que poderiam ser evitados. Neste modesto trabalho, vamos dar apenas algumas sugestões, e não um estudo sobre a Homilética (Arte de Falar em Publico). I -O QUE PREGAR? É a comunicação verbal da Palavra de Deus aos ouvintes. É a transmissão do evangelho de Nosso Senhor Jesus Cristo às pessoas que precisam ouvi-lo. II- QUAL A FINALIDADE DA PREGAÇÃO? É persuadir as pessoas a aceitarem a mensagem da Palavra de Deus para sua salvação (descrentes) ou para seu crescimento espiritual (crentes). Diante d

## Tokenizador

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Dataset, split, tokenização

In [12]:
base_dataset = load_dataset("text", data_files='sample-1gb.txt')
splitted_dataset = base_dataset['train'].train_test_split(test_size = len(base_dataset['train']) // 10, seed=42)
splitted_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-6e39943746455e53/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 225000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 25000
    })
})

In [13]:
tokenized_dataset = splitted_dataset.map(lambda x : tokenizer(x['text'], truncation=True, padding="max_length", max_length=max_seq_length),
                                         batched=True,
                                         num_proc=4,
                                         remove_columns=['text'])

Map (num_proc=4):   0%|          | 0/225000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

### Teste

In [14]:
print(tokenized_dataset)
print(f"First seq: {len(tokenized_dataset['train']['input_ids'][0])} tokens - {tokenized_dataset['train']['input_ids'][0]}")

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 225000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 25000
    })
})
First seq: 256 tokens - [2, 673, 1192, 385, 271, 263, 1455, 242, 3840, 11988, 6044, 354, 1192, 856, 1222, 991, 41, 10744, 1526, 12834, 263, 9074, 5511, 139, 116, 384, 1192, 385, 271, 263, 1455, 242, 3840, 11988, 6044, 354, 1192, 856, 1222, 991, 41, 10744, 1526, 12834, 263, 9074, 5511, 139, 116, 13509, 102, 1242, 9854, 118, 13635, 5107, 1922, 11274, 24913, 2841, 545, 73, 3546, 73, 14420, 6534, 29, 16273, 35, 844, 35, 3414, 384, 118, 181, 3361, 139, 337, 6, 326, 23259, 21045, 139, 3137, 28312, 5563, 29, 38713, 27672, 16845, 263, 10, 12864, 102, 32376, 1464, 687, 6044, 354, 748, 4214, 14156, 11404, 389, 41, 366, 263, 9074, 6510, 117, 507, 2694, 1090, 475, 5563, 29, 364, 364, 257, 16057, 16845, 3286, 11978, 1076, 571, 5037, 1029, 6619, 3840, 364, 1634, 6, 1076, 1140, 119

# Treinamento

In [15]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
model = AutoModelForCausalLM.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir = model_output_dir,
    num_train_epochs = epochs,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    logging_strategy = 'epoch',
    learning_rate = 2e-5,
    weight_decay = 0.01,
    fp16=True
)

trainer = Trainer(
   model = model,
   args = training_args,
   train_dataset = tokenized_dataset['train'],
   eval_dataset = tokenized_dataset['test'],
   data_collator = data_collator
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.697700,2.455794
2,2.429100,2.353614
3,2.342300,2.322958


TrainOutput(global_step=42189, training_loss=2.489714347193581, metrics={'train_runtime': 3920.5519, 'train_samples_per_second': 172.17, 'train_steps_per_second': 10.761, 'total_flos': 8.81860608e+16, 'train_loss': 2.489714347193581, 'epoch': 3.0})

In [16]:
model.save_pretrained(model_save_dir)
tokenizer.save_pretrained(model_save_dir)

('model_save/tokenizer_config.json',
 'model_save/special_tokens_map.json',
 'model_save/vocab.json',
 'model_save/merges.txt',
 'model_save/added_tokens.json',
 'model_save/tokenizer.json')

# Avaliação

## Cálculo da Perplexidade

In [17]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 10.21


## Teste da geração de texto

In [18]:
prompt = 'O café é uma bebida'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenizer(text=prompt)['input_ids']
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos tokens como entrada para o modelo.
    output = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = output['logits'][:, -1, :]  # Usamos apenas o ultimo token da sequencia
    predicted_id = torch.argmax(logits).item()  # extraindo o token de maior probabilidade (greedy decoding)
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt.replace('</s>', ''))

O café é uma bebida que
O café é uma bebida que p
O café é uma bebida que pode
O café é uma bebida que pode ser
O café é uma bebida que pode ser us
O café é uma bebida que pode ser usada
O café é uma bebida que pode ser usada para
O café é uma bebida que pode ser usada para a
O café é uma bebida que pode ser usada para ajud
O café é uma bebida que pode ser usada para ajudar
O café é uma bebida que pode ser usada para ajudar a
O café é uma bebida que pode ser usada para ajudar a man
O café é uma bebida que pode ser usada para ajudar a manter
O café é uma bebida que pode ser usada para ajudar a manter a
O café é uma bebida que pode ser usada para ajudar a manter a l
O café é uma bebida que pode ser usada para ajudar a manter a liga
O café é uma bebida que pode ser usada para ajudar a manter a ligaç
O café é uma bebida que pode ser usada para ajudar a manter a ligação
O café é uma bebida que pode ser usada para ajudar a manter a ligação de
O café é uma bebida que pode ser usada para ajuda

In [19]:
!zip model model_save/*

  adding: model_save/config.json (deflated 49%)
  adding: model_save/generation_config.json (deflated 29%)
  adding: model_save/merges.txt (deflated 53%)
  adding: model_save/pytorch_model.bin (deflated 8%)
  adding: model_save/special_tokens_map.json (deflated 79%)
  adding: model_save/tokenizer_config.json (deflated 73%)
  adding: model_save/tokenizer.json (deflated 72%)
  adding: model_save/vocab.json (deflated 59%)


# Verificação da implementação - overfitting

## Treinamento

In [21]:
training_args = TrainingArguments(
    output_dir = model_output_dir,
    num_train_epochs = 10,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    logging_strategy = 'epoch',
    learning_rate = 2e-5,
    weight_decay = 0.01,
    fp16=True
)

trainer = Trainer(
   model = model,
   args = training_args,
   train_dataset = tokenized_dataset['test'],
   eval_dataset = tokenized_dataset['test'],
   data_collator = data_collator
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.408700,2.246565
2,2.328100,2.171763
3,2.264800,2.109294
4,2.211800,2.058945
5,2.166300,2.014626
6,2.128100,1.979197
7,2.095700,1.949169
8,2.068700,1.927617
9,2.048200,1.912462
10,2.033100,1.906652


TrainOutput(global_step=15630, training_loss=2.175341343894954, metrics={'train_runtime': 1786.3989, 'train_samples_per_second': 139.946, 'train_steps_per_second': 8.749, 'total_flos': 3.2661504e+16, 'train_loss': 2.175341343894954, 'epoch': 10.0})

## Cálculo da Perplexidade

In [22]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 6.73


## Teste da geração de texto

In [30]:
prompt = 'O café é uma bebida'
max_output_tokens = 43
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenizer(text=prompt)['input_ids']
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos tokens como entrada para o modelo.
    output = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = output['logits'][:, -1, :]  # Usamos apenas o ultimo token da sequencia
    predicted_id = torch.argmax(logits).item()  # extraindo o token de maior probabilidade (greedy decoding)
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt.replace('</s>', ''))

O café é uma bebida que
O café é uma bebida que é
O café é uma bebida que é consum
O café é uma bebida que é consumida
O café é uma bebida que é consumida com
O café é uma bebida que é consumida como
O café é uma bebida que é consumida como um
O café é uma bebida que é consumida como um to
O café é uma bebida que é consumida como um tomate
O café é uma bebida que é consumida como um tomate,
O café é uma bebida que é consumida como um tomate, e
O café é uma bebida que é consumida como um tomate, e que
O café é uma bebida que é consumida como um tomate, e que,
O café é uma bebida que é consumida como um tomate, e que, por
O café é uma bebida que é consumida como um tomate, e que, por su
O café é uma bebida que é consumida como um tomate, e que, por sua
O café é uma bebida que é consumida como um tomate, e que, por sua ve
O café é uma bebida que é consumida como um tomate, e que, por sua vez
O café é uma bebida que é consumida como um tomate, e que, por sua vez,
O café é uma bebida que é 